In [1]:
#----------------- Notebook for Hyperdrive & AutoML run ---------------------

In [2]:
##1. Initialise Workspace

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-133731")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E9HC952YL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-133731
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-133731


In [4]:
##2. Create AmlCompute Cluster

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster1"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
##3. Create Project Directory
##4. Create an estimator using SKLEARN
##5. Create hyperdrive config

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
#--from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "max_iter": uniform(50, 100),
        "--C": uniform(0.5,1)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
                    compute_target=cpu_cluster,
                    entry_script='train.py'
                   )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                  hyperparameter_sampling=ps, 
                  policy= policy,
                  primary_metric_name="accuracy",
              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                  max_total_runs=20,
                  max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())
model = best_run.register_model(model_name='udacity-project', model_path='outputs/model.joblib')

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3692402e-d729-4a9e-9f02-4bdf4c4ae271
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3692402e-d729-4a9e-9f02-4bdf4c4ae271?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-133731/workspaces/quick-starts-ws-133731

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-06T23:40:58.694430][API][INFO]Experiment created<END>\n""<START>[2021-01-06T23:40:59.272305][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-06T23:40:59.437361][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-06T23:41:00.7905039Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>
{
  "azureml-logs/20_image_build_log.txt": "https://mlstrg133731.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3692402e-d729-4a9e-9f02-4bdf4c4ae271_preparation/azureml-logs/20_image_build_l

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Environment preparation run marked as Failed by execution service due to UserError.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Environment preparation run marked as Failed by execution service due to UserError.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###